# Importing Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Test set Image Processing

In [ ]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    'test',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)
class_name = validation_set.class_names
print(class_name)

# Loading Model

In [ ]:
cnn = tf.keras.models.load_model('trained_plant_disease_model.keras')

## Visualising and Performing Prediction on Single image

In [ ]:
import cv2

#Test Image Visualization


image_path = r'D:\00_google_classroom\tensorflow - Copy\test\Tomato_Bacterial_spot\0b13b997-9957-4029-b2a4-ef4a046eb088___UF.GRC_BS_Lab Leaf 0595.jpg'
# Reading an image in default mode
img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f"Image not found at {image_path}")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Converting BGR to RGB
# Displaying the image 
plt.imshow(img)
plt.title('Test Image')
plt.xticks([])
plt.yticks([])
plt.show()

## Testing Model

In [ ]:
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr]) / 255.0  # Normalize pixel values
predictions = cnn.predict(input_arr)

In [ ]:
print(predictions)

In [ ]:
result_index = np.argmax(predictions) #Return index of max element
print(result_index)

In [ ]:
actual_class = image_path.split("\\")[-2]  # Extract actual class from the image path
predicted_class = class_name[result_index]
confidence_score = np.max(predictions)
accuracy_score = confidence_score * 100

plt.imshow(img)
plt.title(f"Actual: {actual_class}\nPredicted: {predicted_class}\nConfidence: {accuracy_score:.2f}%")
plt.xticks([])
plt.yticks([])
plt.show()